In [1]:
import polars as pl
import polars.selectors as cs
import re


In [2]:
from operator import add

In [3]:
(health_survey := pl.read_csv("./data/health_survey (1).csv")
)

ID,F1,F5,F2,F1.1,F2.1,F6,F4,F3,F5.1,F1.2,F2.2,F6.1,F2.3,F4.1,F2.4,F5.2,F2.5,F6.2,F1.3,F2.6,F5.3,F4.2,F2.7,F3.1,F2.8,F5.4,F3.2,F1.4,F3.3,F1.5,F5.5,F6.3,F1.6,F5.6,F2.9,F3.4,F4.3,F2.10,F1.7,F6.4,F4.4,F5.7,F3.5,F2.11
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""
3,"""Strongly Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""",null,"""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Disagree""","""Somewhat Agree"""
4,"""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewh

In [4]:
(Rev_C := pl.read_csv("./data/ReverseCodingItems (1).csv")
)

Question,Construct,Question # on Qualtrics Survey,Needs Reverse Coding?,Column Name
str,i64,i64,str,str
"""In the future, I plan to parti…",1,1,"""No""","""F1"""
"""Individuals are responsible fo…",5,2,"""Yes""","""F5"""
"""When tryng to understand the p…",2,3,"""No""","""F2"""
"""I plan to become involved in m…",1,4,"""No""","""F1.1"""
"""I can communicate well with ot…",2,5,"""No""","""F2.1"""
…,…,…,…,…
"""I prefer the company of people…",6,40,"""Yes""","""F6.4"""
"""I have the ability to lead a g…",4,41,"""No""","""F4.4"""
"""It is important to me that equ…",5,42,"""No""","""F5.7"""


In [5]:
rename_dict = {
    q: f"{q}_rev" if flag == "Yes" else f"{q}_reg"
    for q, flag in zip(Rev_C["Column Name"], Rev_C["Needs Reverse Coding?"])
}


In [6]:
health_survey_renamed = health_survey.rename(rename_dict)  
health_survey_renamed 

ID,F1_reg,F5_rev,F2_reg,F1.1_reg,F2.1_reg,F6_rev,F4_reg,F3_reg,F5.1_reg,F1.2_reg,F2.2_reg,F6.1_reg,F2.3_reg,F4.1_rev,F2.4_reg,F5.2_rev,F2.5_reg,F6.2_reg,F1.3_reg,F2.6_reg,F5.3_reg,F4.2_reg,F2.7_reg,F3.1_reg,F2.8_reg,F5.4_reg,F3.2_reg,F1.4_reg,F3.3_reg,F1.5_reg,F5.5_rev,F6.3_rev,F1.6_reg,F5.6_rev,F2.9_reg,F3.4_reg,F4.3_rev,F2.10_reg,F1.7_reg,F6.4_rev,F4.4_reg,F5.7_reg,F3.5_reg,F2.11_reg
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""
3,"""Strongly Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""",null,"""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Disagree""","""Somewhat Agree"""
4,"""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Strongly Agree""","""Somewhat Disagree""","""S

In [7]:
(reg := {'Strongly Agree':5,
         'Somewhat Agree':4,
         'Neither Agree nor Disagree':3,
         'Somewhat Disagree':2,
         'Strongly Disagree':1
        }
)

{'Strongly Agree': 5,
 'Somewhat Agree': 4,
 'Neither Agree nor Disagree': 3,
 'Somewhat Disagree': 2,
 'Strongly Disagree': 1}

In [8]:
(rev := {'Strongly Agree':1,
         'Somewhat Agree':2,
         'Neither Agree nor Disagree':3,
         'Somewhat Disagree':4,
         'Strongly Disagree':5
        }
)

{'Strongly Agree': 1,
 'Somewhat Agree': 2,
 'Neither Agree nor Disagree': 3,
 'Somewhat Disagree': 4,
 'Strongly Disagree': 5}

In [10]:
(health_survey_numeric := health_survey_renamed
 .with_columns([
        cs.ends_with("_reg").replace(reg).cast(pl.Int64()).name.suffix("_num"),
        cs.ends_with("_rev").replace(rev).cast(pl.Int64()).name.suffix("_num")
        ])
 .drop(cs.string())
 .with_columns(pl.reduce(add, cs.contains('F1')).alias('F1_total'),
               pl.reduce(add, cs.contains('F2')).alias('F2_total'),
               pl.reduce(add, cs.contains('F3')).alias('F3_total'),
               pl.reduce(add, cs.contains('F4')).alias('F4_total'),
               pl.reduce(add, cs.contains('F5')).alias('F5_total'),
               pl.reduce(add, cs.contains('F6')).alias('F6_total'))
 .drop(cs.contains('num'))
)


ID,F1_total,F2_total,F3_total,F4_total,F5_total,F6_total
i64,i64,i64,i64,i64,i64,i64
1,31,48,20,17,28,18
2,31,47,19,17,27,20
3,36,null,19,18,32,17
4,32,54,12,15,30,16
5,37,47,22,19,36,19
…,…,…,…,…,…,…
260,32,50,22,18,34,22
261,31,48,24,19,28,null
262,30,47,20,23,33,19


In [11]:
(health_survey_numeric := health_survey_renamed
 .with_columns([
    cs.ends_with("_reg").replace_strict(reg, default=3).cast(pl.Int64).name.suffix("_num"),
    cs.ends_with("_rev").replace_strict(rev, default=3).cast(pl.Int64).name.suffix("_num"),
 ])
 .drop(cs.string())
.with_columns([
    pl.reduce(add, cs.contains(f'F{i}')).alias(f'F{i}_total')
    for i in range(1, 7)
])
.drop(cs.ends_with("_num"))
)

ID,F1_total,F2_total,F3_total,F4_total,F5_total,F6_total
i64,i64,i64,i64,i64,i64,i64
1,31,48,20,17,28,18
2,31,47,19,17,27,20
3,36,49,19,18,32,17
4,32,54,12,15,30,16
5,37,47,22,19,36,19
…,…,…,…,…,…,…
260,32,50,22,18,34,22
261,31,48,24,19,28,18
262,30,47,20,23,33,19
